In [2]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import Order, OrderStatus
from tagger.viewsets.order.change_status import ChangeStatusSerializer, _change_status
from django.contrib.auth.models import User
from tagger.models.inventory import Inventory, InventoryStatus

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd

def split_codes(raw_codes):
    raw_codes = raw_codes.replace("_", "")
    codes = []
    cursor = 0
    while cursor < len(raw_codes):
        codes.append(raw_codes[cursor: cursor + 5])
        cursor = cursor + 5
    return codes

In [3]:
with open("../Downloads/확장주문검색_20211116_152516.xlsx", "rb") as f:
    df = pd.read_excel(f, dtype=str)
    df.dropna(subset=["주문번호"], inplace=True)
    for _, row in df.iterrows():
        print(split_codes(row["주문번호"]), row["판매처 상품명"], row["수령자이름"])

['CANMD'] 테일러드 더블 코트 김은지
['YR83V'] 체크패턴 벨티드 코트(멜란지 그레이) 김효영
['LFRTQ'] 밍크 퍼 배색 코트 박은숙
['NARRW'] 벨티드 플랩 코트(카멜) 조정수
['YK4FT'] [로렌 랄프로렌] 서플리스 데님 미디 드레스 한미나
['3NYGX'] [MEN] (7 colors) 울 캐시미어 스웨터 황태실


In [9]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    tracking_no = f"{row['택배사']} {row['송장번호']}"
    tracking_url = f"https://search.naver.com/search.naver?query={tracking_no}"
    for code in codes:
        order = Order.get(code)
        s = ChangeStatusSerializer(data={
            "status": OrderStatus.DELIVERY_STARTED,
            "delivery_tracking_number": tracking_no,
            "delivery_tracking_url": tracking_url,
        }, instance=order)
        s.is_valid(raise_exception=True)
        _change_status(s, order, User.objects.get(username="backoffice_server"), id=str(order.id))
        print(f"https://office.alloff.co/orders/{code}", order.payment.buyername, order.payment.name)

https://office.alloff.co/orders/CANMD 김은지 테일러드 더블 코트 
https://office.alloff.co/orders/YR83V 김효영  체크패턴 벨티드 코트(멜란지 그레이)
https://office.alloff.co/orders/LFRTQ 박은숙  밍크 퍼 배색 코트 
https://office.alloff.co/orders/NARRW 조정수 벨티드 플랩 코트(카멜) 
https://office.alloff.co/orders/YK4FT 한미나 [로렌 랄프로렌] 서플리스 데님 미디 드레스
https://office.alloff.co/orders/3NYGX 황태실 [MEN] (7 colors) 울 캐시미어 스웨터


In [10]:
for _, row in df.iterrows():
    codes = split_codes(row["주문번호"])
    for code in codes:
        order = Order.get_by_code(code)
        if order is not None:
            inventories = Inventory.objects.filter(status=InventoryStatus.SHIPPING_PENDING, out_order_id=str(order.id)).all()
            for i in inventories:
                print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
                i.status = InventoryStatus.SHIPPED
                i.save()

CANMD (618d019d26685ff051756218) - 테일러드 더블 코트 has been shipped!
YR83V (618882f226685ff051755a6c) -  체크패턴 벨티드 코트(멜란지 그레이) has been shipped!
LFRTQ (618dce1826685ff05175634a) - 밍크 퍼 배색 코트  has been shipped!
NARRW (6187f34826685ff051755a02) - 벨티드 플랩 코트(카멜)  has been shipped!
YK4FT (617bed3017fc2e5a1f9eae91) - [로렌 랄프로렌] 서플리스 데님 미디 드레스 has been shipped!
3NYGX (617bcaf817fc2e5a1f9eade4) - [MEN] (7 colors) 울 캐시미어 스웨터 has been shipped!


In [7]:
for i in Inventory.objects.filter(out_order_id__in=[
    "618bdd3d26685ff05175601e",
    "617a9edb17fc2e5a1f9eabe7",
    "618b6a9f26685ff051755f2e",
    "61862bf826685ff0517556c7",
    "6183b47226685ff0517551b1",
    "618a4ca126685ff051755d58",
    "6180d93017fc2e5a1f9eb4ee",
    "618659d826685ff051755731",
    "618265fc26685ff051754f33",
    "6180619517fc2e5a1f9eb42f",
    "6182698926685ff051754f47",
]):
    order = Order.get(i.out_order_id)
    print(f"""{order.code} ({order.id}) - {i.product_name} has been shipped!""")
    i.status = InventoryStatus.SHIPPED
    i.save()

ORD-WAHUH (618bdd3d26685ff05175601e) - 캐시미어 니트 롱 드레스 has been shipped!
ORD-LWH3Y (617a9edb17fc2e5a1f9eabe7) - [택 제거 상품] Teca has been shipped!
ORD-KRJUB (6180619517fc2e5a1f9eb42f) - Torto sweatshirt(네이비) has been shipped!
ORD-X8K9D (6180d93017fc2e5a1f9eb4ee) - [로렌 랄프로렌] 브로드클로스 블루종 슬리브 셔츠 has been shipped!
ORD-PAKK6 (6183b47226685ff0517551b1) - 네오플랜 블록 구스다운 점퍼(DARK GRAY)  has been shipped!
ORD-GKYKH (61862bf826685ff0517556c7) - Ashurst quilted jacket(블랙) has been shipped!
ORD-GL7NK (618265fc26685ff051754f33) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-VERXH (6182698926685ff051754f47) - 핸드메이드 캐시미어 코트  has been shipped!
ORD-BGH46 (618659d826685ff051755731) - [빈폴스포츠] 투웨이 점퍼&조끼 다운 베스트(화이트)  has been shipped!
ORD-9YUM9 (618a4ca126685ff051755d58) - 벨티드 구스 패딩(화이트 핑크)  has been shipped!
ORD-8XRFR (618b6a9f26685ff051755f2e) - 프레이드 트위드 드레스(다크네이비) has been shipped!


True